In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import transformers
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-931380d0e19583fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
dataset['train'][0].keys()

dict_keys(['id', 'dialogue', 'summary', 'topic'])

In [35]:
for i in range(1,3):
    print(f"Example# {i}\n{'-'*100}")
    print(f"Input Dialogue:\n{dataset['test'][i]['dialogue']}\n")
    print(f"Summary:\n{dataset['test'][i]['summary']}")
    print(f"{'-'*100}\n")

Example# 1
----------------------------------------------------------------------------------------------------
Input Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
------

In [6]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [37]:
encoded = tokenizer(text = "That sounds great. Thanks.", return_tensors='pt')

decoded = tokenizer.decode(token_ids= encoded['input_ids'][0],
                           skip_special_tokens=True)

print(f'Encoded Sentence: {encoded["input_ids"][0]}\n')
print(f'Decoded Sentence: {decoded}\n')

Encoded Sentence: tensor([ 466, 2993,  248,    5, 1333,    5,    1])

Decoded Sentence: That sounds great. Thanks.



# INITIAL MODEL EVALUATION - NO PROMPT ENGINEERING

In [50]:
for num, i in enumerate([10,50]):
    input_dialogue = dataset['test'][i]['dialogue']
    summary = dataset['test'][i]['summary']
    
    input_tokens = tokenizer(text=input_dialogue,
                             return_tensors='pt')
    output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                   max_new_tokens=50)
    
    output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                      skip_special_tokens=True)
    
    print(f"Example# {num+1}\n{'-'*100}")
    
    print(f"Input Dialogue:\n{dataset['test'][i]['dialogue']}\n")
    print(f"Summary:\n{dataset['test'][i]['summary']}\n")
    print(f"Model Generated Summary:\n{output_summary}")
    
    print(f"{'-'*100}\n")

Example# 1
----------------------------------------------------------------------------------------------------
Input Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

Summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

Model Generated Summary:
Brian, thank you for coming to our party.
-----------------------------------

Responses are not that good.
- In the first example it seems like model is generating the next sentence for the dialogue.
- Resposne in the second example does not fit to our need at all.
- We can consider prompt engineering for summary generation.

# PROMPT ENGINEERING

#### ZERO-SHOT INFERENCE

In this type of in-context learning, we can consider adding an instruction in the prompt for the task to be accomplished. 

In [51]:
for num, i in enumerate([10,50]):
    input_dialogue = dataset['test'][i]['dialogue']
    summary = dataset['test'][i]['summary']
    
    prompt = f"Give a summary of the following dialogue:\n{input_dialogue}\nSummary:"
    input_tokens = tokenizer(text=prompt,
                             return_tensors='pt')
    output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                   max_new_tokens=50)
    
    output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                      skip_special_tokens=True)
    
    print(f"Example# {num+1}\n{'-'*100}")
    
    print(f"Prompt:\n{prompt}\n")
    print(f"Summary:\n{dataset['test'][i]['summary']}\n")
    print(f"Model Generated Summary:\n{output_summary}")
    
    print(f"{'-'*100}\n")

Example# 1
----------------------------------------------------------------------------------------------------
Prompt:
Give a summary of the following dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Summary:

Summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

Model Generated Summary:
Brian's birthday is coming up.
---

The model is understanding the context well but unable to precise conslusion.

## ZERO SHOT INFERENCE WITH PRE-BUILT FLAN-T5 PROMPT

Let's try one of the prompt from the FLAN-T5 prompt templates and see if model could pick up on the nuance of the dialogue.

In [54]:
for num, i in enumerate([10,50]):
    input_dialogue = dataset['test'][i]['dialogue']
    summary = dataset['test'][i]['summary']
    
    prompt = f"Dialogue:\n{input_dialogue}\nWhat was going on?"
    input_tokens = tokenizer(text=prompt,
                             return_tensors='pt')
    output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                   max_new_tokens=50)
    
    output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                      skip_special_tokens=True)
    
    print(f"Example# {num+1}\n{'-'*100}")
    
    print(f"Prompt:\n{prompt}\n")
    print(f"Summary:\n{dataset['test'][i]['summary']}\n")
    print(f"Model Generated Summary:\n{output_summary}")
    
    print(f"{'-'*100}\n")

Example# 1
----------------------------------------------------------------------------------------------------
Prompt:
Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
What was going on?

Summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

Model Generated Summary:
Brian's birthday is coming up.
-------------------------

Unfortunetly, the model behaviour stays the same. We could consider improving it using the one-shot inference.m

## ONE SHOT INFERENCE

In [63]:
test_example_index = 10
test_evaluation_index = 50
example_dialogue = dataset['test'][test_example_index]['dialogue']
input_dialogue = dataset['test'][test_evaluation_index]['dialogue']
example_summary = dataset['test'][test_example_index]['summary']
summary = dataset['test'][test_evaluation_index]['summary']

prompt = f"""
Dialogue:\n{example_dialogue}
What was going on?\n{example_summary}
Dialogue:\n{input_dialogue}
What was going on? """

input_tokens = tokenizer(text=prompt,
                         return_tensors='pt')
output_tokens = model.generate(inputs=input_tokens['input_ids'],
                               max_new_tokens=50)

output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                  skip_special_tokens=True)

print(f"Prompt:\n{prompt}\n")
print(f"{'-'*100}\n")

print(f"Summary:\n{dataset['test'][test_evaluation_index]['summary']}\n")
print(f"{'-'*100}\n")

print(f"Model Generated Summary:\n{output_summary}")

print(f"{'-'*100}\n")

Prompt:

Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
What was going on?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.
Dialogue:
#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our frien

We can't see much better performance here as well. Let's change the prompt a bit and see how it performs.

In [71]:
test_example_index = 10
test_evaluation_index = 50
example_dialogue = dataset['test'][test_example_index]['dialogue']
input_dialogue = dataset['test'][test_evaluation_index]['dialogue']
example_summary = dataset['test'][test_example_index]['summary']
summary = dataset['test'][test_evaluation_index]['summary']

prompt = f"""
Give a summary of the following dialogue:\n{example_dialogue}
Summary:\n{example_summary}
Give a summary of the following dialogue:\n{input_dialogue}
Summary:"""

input_tokens = tokenizer(text=prompt,
                         return_tensors='pt')
output_tokens = model.generate(inputs=input_tokens['input_ids'],
                               max_new_tokens=50)

output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                  skip_special_tokens=True)

print(f"Prompt:\n{prompt}\n")
print(f"{'-'*100}\n")

print(f"Summary:\n{dataset['test'][test_evaluation_index]['summary']}\n")
print(f"{'-'*100}\n")

print(f"Model Generated Summary:\n{output_summary}")

print(f"{'-'*100}\n")

Prompt:

Give a summary of the following dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.
Give a summary of the following dialogue:
#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don'

The model still not generate summary upto the expectations.

## FEW SHOT INFERENCE

In [79]:
test_example_indices = [2, 20]
test_evaluation_index = 50
input_dialogue = dataset['test'][test_evaluation_index]['dialogue']
summary = dataset['test'][test_evaluation_index]['summary']

prompt = ""
for i in test_example_indices:
    prompt += f"""
    Dialogue:\n{dataset['test'][i]['dialogue']}
    What was going on?\n{dataset['test'][i]['summary']}
    """
prompt += f"""Dialogue:\n{input_dialogue}
What was going on?"""

input_tokens = tokenizer(text=prompt,
                         return_tensors='pt')
output_tokens = model.generate(inputs=input_tokens['input_ids'],
                               max_new_tokens=50)

output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                  skip_special_tokens=True)

print(f"Prompt:\n{prompt}\n")
print(f"{'-'*100}\n")

print(f"Summary:\n{dataset['test'][test_evaluation_index]['summary']}\n")
print(f"{'-'*100}\n")

print(f"Model Generated Summary:\n{output_summary}")

print(f"{'-'*100}\n")

Prompt:

    Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please con

In [80]:
len(input_tokens[0])

848

In few shot inference, the model do not generate any better. We can conclude one-shot learnign to be more effective than any othere here.

Another thing we can do is to change the configuration parameters of the `model.generate()` fucntion to see variability in the output.

# Configuration Parameters

In [200]:
import pandas as pd
import itertools

In [222]:
results = pd.DataFrame(columns = ['In-Context Learning Type',
                                  'prompt_template',
                                  'max_new_tokens', 'do_sample', 
                                  'temperature','Human Generated Summary',
                                  'Model Generate Summary'])

In [229]:
dic = {
  'In-Context Learning Type':1,
  'prompt_template':2,
  'max_new_tokens':3, 'do_sample':3, 
  'temperature':3,'Human Generated Summary':2,
  'Model Generate Summary':2
}

In [242]:
def create_prompt(inference_type:str,
                  prompt_template=None,
                  testing_indices=[10],
                  evaluation_index=100):
    
    input_dialogue = dataset['test'][evaluation_index]['dialogue']
    
    if inference_type == 'zero-shot':
        return input_dialogue
    elif inference_type == 'one-shot':
        testing_indices = [testing_indices[0]]
    
    prompt_start = prompt_template.split("{")[0]
    prompt_end = prompt_template.split("}")[1]
    prompt = ""
    for i in testing_indices:
        prompt += f"{prompt_start}\n{dataset['test'][i]['dialogue']}\n{prompt_end}\n{dataset['test'][i]['summary']}\n"
        prompt +="\n"
    
    prompt += f"{prompt_start}\n{input_dialogue}\n{prompt_end}"
    
    return prompt

In [181]:
template_1 = "Dialogue:{dialogue}\nWhats going on?"
template_2 = "Summarize the following conversation:{dialogue}\nSummary:"
template_3 = "Provide summary:{dialogue}\nSummary:"

templates = [template_1, template_2, template_3]

In [203]:
params = {
    'max_new_tokens':[10,50],
    'do_sample':[True],
    'temperature':[None, 0.1, 0.5, 1.0]
}
param_combinations = list(itertools.product(*params.values()))

**Let's first re-generate the zero-shot examples.**

In [244]:
evaluation_index = 100
prompt = create_prompt(inference_type="zero-shot",
                       evaluation_index=evaluation_index)

input_tokens = tokenizer(text=prompt,
                         return_tensors='pt')
actual_summary = dataset['test'][evaluation_index]['summary']

for combination in param_combinations:
    param_dict = dict(zip(params.keys(), combination))
    output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                   generation_config=GenerationConfig(
                                       max_new_tokens=param_dict['max_new_tokens'],
                                       do_sample=param_dict['do_sample'],
                                       temperature=param_dict['temperature']
                                   ))
    output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                      skip_special_tokens=True)

    results = results.append({'In-Context Learning Type':'zero-shot',
                    'prompt_template':"-",
                    'max_new_tokens':param_dict['max_new_tokens'], 
                    'do_sample':param_dict['do_sample'],
                    'temperature':param_dict['temperature'],
                    'Human Generated Summary':actual_summary,
                    'Model Generate Summary':output_summary
                   }, ignore_index=True)

/tmp/ipykernel_21/1772620600.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'zero-shot',
/tmp/ipykernel_21/1772620600.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'zero-shot',
/tmp/ipykernel_21/1772620600.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'zero-shot',
/tmp/ipykernel_21/1772620600.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'zero-shot',
/tmp/ipykernel_21/1772620600.py:20: FutureWarning: T

**Now we will generate one shot inferences and then move on to few-shot inferences.**

In [245]:
evaluation_index = 100
for template in templates:
    prompt = create_prompt(inference_type="one-shot",
                           prompt_template=template,
                           testing_indices=[10],
                           evaluation_index=evaluation_index)

    input_tokens = tokenizer(text=prompt,
                             return_tensors='pt')
    actual_summary = dataset['test'][evaluation_index]['summary']
    
    for combination in param_combinations:
        param_dict = dict(zip(params.keys(), combination))
        output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                       generation_config=GenerationConfig(
                                           max_new_tokens=param_dict['max_new_tokens'],
                                           do_sample=param_dict['do_sample'],
                                           temperature=param_dict['temperature']
                                       ))
        output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                          skip_special_tokens=True)
        
        results = results.append({'In-Context Learning Type':'one-shot',
                        'prompt_template':template,
                        'max_new_tokens':param_dict['max_new_tokens'], 
                        'do_sample':param_dict['do_sample'],
                        'temperature':param_dict['temperature'],
                        'Human Generated Summary':actual_summary,
                        'Model Generate Summary':output_summary
                       }, ignore_index=True)

/tmp/ipykernel_21/720667955.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'one-shot',
/tmp/ipykernel_21/720667955.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'one-shot',
/tmp/ipykernel_21/720667955.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'one-shot',
/tmp/ipykernel_21/720667955.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'one-shot',
/tmp/ipykernel_21/720667955.py:23: FutureWarning: The frame.

In [246]:
evaluation_index = 100
for template in templates:
    prompt = create_prompt(inference_type="few-shot",
                           prompt_template=template,
                           testing_indices=[10,60],
                           evaluation_index=evaluation_index)

    input_tokens = tokenizer(text=prompt,
                             return_tensors='pt')
    actual_summary = dataset['test'][evaluation_index]['summary']
    
    for combination in param_combinations:
        param_dict = dict(zip(params.keys(), combination))
        output_tokens = model.generate(inputs=input_tokens['input_ids'],
                                       generation_config=GenerationConfig(
                                           max_new_tokens=param_dict['max_new_tokens'],
                                           do_sample=param_dict['do_sample'],
                                           temperature=param_dict['temperature']
                                       ))
        output_summary = tokenizer.decode(token_ids=output_tokens[0],
                                          skip_special_tokens=True)
        
        results = results.append({'In-Context Learning Type':'few-shot',
                        'prompt_template':template,
                        'max_new_tokens':param_dict['max_new_tokens'], 
                        'do_sample':param_dict['do_sample'],
                        'temperature':param_dict['temperature'],
                        'Human Generated Summary':actual_summary,
                        'Model Generate Summary':output_summary
                       }, ignore_index=True)

/tmp/ipykernel_21/3121368368.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'few-shot',
/tmp/ipykernel_21/3121368368.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'few-shot',
/tmp/ipykernel_21/3121368368.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'few-shot',
/tmp/ipykernel_21/3121368368.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'In-Context Learning Type':'few-shot',
/tmp/ipykernel_21/3121368368.py:23: FutureWarning: The f

In [263]:
results.head(2)

,In-Context Learning Type,prompt_template,max_new_tokens,do_sample,temperature,Human Generated Summary,Model Generate Summary
0,1,2,3,3,3,2,2
1,zero-shot,Dialogue:{dialogue}\nWhats going on?,10,True,None,#Person1# and Mike have a disagreement on how ...,#Person1#: This is a


In [287]:
zero_shot_inf = results[results['In-Context Learning Type']=='zero-shot']
for i in zero_shot_inf[20:].iterrows():
    print(f"max_new_tokens:\t{i[1]['max_new_tokens']}")
    print(f"temperature:\t{i[1]['temperature']}")
    print(f"Human Generated Summary:\n{i[1]['Human Generated Summary']}")
    print("\n")
    print(f"Model Generate Summary:\n{i[1]['Model Generate Summary']}")
    print(f"{'*'*100}\n")

max_new_tokens:	50
temperature:	None
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
#Person2#: Mike, how the others react, and try to understand them.
****************************************************************************************************

max_new_tokens:	50
temperature:	0.1
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
#Person1: I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to hear that. I'm sorry to
****************************************************************************************************

max_new_tokens:	50
temperature:	0.5
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that 

## ONE SHOT INFERENCE RESULTS

In [290]:
one_shot_inf = results[results['In-Context Learning Type']=='one-shot']
for i in one_shot_inf.iterrows():
    print(f"max_new_tokens:\t{i[1]['max_new_tokens']}")
    print(f"temperature:\t{i[1]['temperature']}")
    print(f"prompt_template:\t{i[1]['prompt_template']}")
    print(f"Human Generated Summary:\n{i[1]['Human Generated Summary']}")
    print("\n")
    print(f"Model Generate Summary:\n{i[1]['Model Generate Summary']}")
    print(f"{'*'*100}\n")

max_new_tokens:	10
temperature:	None
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
People discuss how Mike responded to a joke and
****************************************************************************************************

max_new_tokens:	10
temperature:	0.1
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
The two people are trying to figure out how to
****************************************************************************************************

max_new_tokens:	10
temperature:	0.5
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how 

## FEW SHOT INFERENCE RESULTS

In [291]:
few_shot_inf = results[results['In-Context Learning Type']=='few-shot']
for i in one_shot_inf.iterrows():
    print(f"max_new_tokens:\t{i[1]['max_new_tokens']}")
    print(f"temperature:\t{i[1]['temperature']}")
    print(f"prompt_template:\t{i[1]['prompt_template']}")
    print(f"Human Generated Summary:\n{i[1]['Human Generated Summary']}")
    print("\n")
    print(f"Model Generate Summary:\n{i[1]['Model Generate Summary']}")
    print(f"{'*'*100}\n")

max_new_tokens:	10
temperature:	None
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
People discuss how Mike responded to a joke and
****************************************************************************************************

max_new_tokens:	10
temperature:	0.1
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.


Model Generate Summary:
The two people are trying to figure out how to
****************************************************************************************************

max_new_tokens:	10
temperature:	0.5
prompt_template:	Dialogue:{dialogue}
Whats going on?
Human Generated Summary:
#Person1# and Mike have a disagreement on how 